In [1]:
import spacy
from spacy import displacy
from spacy.matcher import Matcher
import re
import time
import pdfplumber
from openpyxl import Workbook

In [ ]:

def generar_diccionario_ampliaciones(file):
    diccionario_obras_ampliacion = {}

    paginas = [1, 2]

    with pdfplumber.open(file) as pdf:
        text = ""
        for i in paginas:
            text += pdf.pages[i].extract_text()

        matches = re.finditer(r'((Ampliación en S/E).*?)(?=\n\S)', text, re.DOTALL)
        for match in matches:
            line = match.group(0)
            if not re.match(r".*[0-9]{2}$", line):
                print("Título largo: ", line)
                titulo = input("Ingrese el título del proyecto según el índice del pdf: ")
                pag_inicio = int(input("Ingrese la página de inicio del proyecto según el índice del pdf: ")) - 1
                pag_final = pag_inicio + 3

            else:
                match = re.match(r'^(.*?)\s+(\d+)$', line)
                if match:
                    titulo = match.group(1).replace(".", "")
                    pag_inicio = int(match.group(2)) - 1
                    pag_final = pag_inicio + 3

            
            diccionario_obras_ampliacion[titulo] = (pag_inicio, pag_final)


    return diccionario_obras_ampliacion




In [ ]:
def extraer_texto_entre_delimitadores_v2(texto, delimitador_inicial, delimitador_final):
    pattern = re.compile(rf"{re.escape(delimitador_inicial)}(.*?){re.escape(delimitador_final)}.*?\.", re.DOTALL)
    match = pattern.search(texto)
    return match.group(0) if match else "ERROR EXTRAYENDO TEXTO"


In [2]:
from funciones_extra import generar_diccionario_ampliaciones, generar_diccionario_descripciones_amp, extraer_texto_entre_delimitadores_v2
from proyecto_ampliacion import Proyecto_ampliacion

c:\Users\Matías Fernandez\Desktop\Ampliaciones\src\Ampliaciones\proyecto_ampliacion.py:696: SyntaxWarning: invalid escape sequence '\.'
  pattern = re.compile(f"{delimitador_inicial}(.*?{delimitador_final}.*?)\.", re.DOTALL)
c:\Users\Matías Fernandez\Desktop\Ampliaciones\src\Ampliaciones\proyecto_ampliacion.py:766: SyntaxWarning: invalid escape sequence '\p'
  file = "Informes\plan_expansion_final_2023.pdf"


In [3]:
file = "plan_expansion_final_2023.pdf"
diccionario = generar_diccionario_ampliaciones(file)
dic_proyectos = generar_diccionario_descripciones_amp(file, diccionario)

In [ ]:

def extraer_texto(proyecto_str):
    # Definimos las frases de inicio y fin
    frase_inicio = "El proyecto consiste en"
    frase_fin = "El proyecto incluye todas las obras"
    
    # Utilizamos una expresión regular para encontrar el texto entre las frases
    patron = re.compile(re.escape(frase_inicio) + r'(.*?)(?=' + re.escape(frase_fin) + ')', re.DOTALL)
    
    # Buscamos el patrón en el string de entrada
    coincidencia = patron.search(proyecto_str)
    
    # Si encontramos una coincidencia, la devolvemos, de lo contrario devolvemos None
    if coincidencia:
        return coincidencia.group(0).strip()
    else:
        return None

In [4]:
def ejecutar_analisis_v3(dic_proyectos):
    dic_proyectos_con_excepciones = {}
    for titulo, descripcion in dic_proyectos.items():
        proyecto = Proyecto_ampliacion(titulo, descripcion)
        proyecto.procesar_descripcion()


In [5]:
ejecutar_analisis_v3(dic_proyectos)

Nombre del proyecto: Ampliación en S/E Cóndores 110 kV (BP+BT) 
Tipo de proyecto: Ampliación
Resumen del proyecto: El proyecto consiste en la ampliación de las barras e instalaciones comunes del patio de 110 kV de la subestación Cóndores, cuya configuración corresponde a barra principal con barra de transferencia, para dos nuevas posiciones, de manera de permitir la conexión de la obra “Nueva S/E Alto Molle y nueva línea 2x110 kV Alto Molle – Cóndores”.
Valor de inversión: 1.579.763 dólares
Entrada en operación: a más tardar, dentro de los 54 meses siguientes a la fecha de publicación en el Diario Oficial del respectivo decreto


Nombre del proyecto: Ampliación en S/E Nueva Cardones 220 kV (IM) 
Tipo de proyecto: Ampliación
Resumen del proyecto: El proyecto consiste en la ampliación del galpón e instalaciones comunes del patio de 220 kV de la subestación Nueva Cardones, cuya configuración corresponde a interruptor y medio, para cuatro nuevas diagonales, de manera de permitir la conexió